In [18]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras import backend as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense,Dropout
from keras.datasets import mnist
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop, Adam
import numpy as np
import matplotlib.pyplot as plt

In [2]:

from subprocess import check_output
print(check_output(["ls", "./data"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

test.csv
train.csv



### Load Train and Test Data:

In [3]:
# create the training & test sets, skipping the header row with [1:]
train = pd.read_csv("./data/train.csv")
print(train.shape)
train.head()

(42000, 785)


,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
test= pd.read_csv("./data/test.csv")
print(test.shape)
test.head()

(28000, 784)


,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X_train = train.drop(labels = ["label"],axis = 1) 
print(X_train)

       pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0           0       0       0       0       0       0       0       0       0   
1           0       0       0       0       0       0       0       0       0   
2           0       0       0       0       0       0       0       0       0   
3           0       0       0       0       0       0       0       0       0   
4           0       0       0       0       0       0       0       0       0   
5           0       0       0       0       0       0       0       0       0   
6           0       0       0       0       0       0       0       0       0   
7           0       0       0       0       0       0       0       0       0   
8           0       0       0       0       0       0       0       0       0   
9           0       0       0       0       0       0       0       0       0   
10          0       0       0       0       0       0       0       0       0   
11          0       0       

In [6]:
Y_train = train['label'].values.astype('int32') # only labels i.e targets digits
X_test = test
print(Y_train)

[1 0 1 ..., 7 6 9]


In [7]:
print("Shape of Training dataset Images:",X_train.shape)
print("Shape of Training dataset labels:",Y_train.shape)

Shape of Training dataset Images: (42000, 784)
Shape of Training dataset labels: (42000,)


In [8]:
K.set_image_dim_ordering("th")

In [9]:

X_train /= 255.0
X_test /= 255.0

In [10]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,1,28,28)
X_test = X_test.values.reshape(-1,1,28,28)

In [11]:
print(X_train.shape)
print(X_test.shape)

(42000, 1, 28, 28)
(28000, 1, 28, 28)


In [12]:
print("Training Sample Shape:",X_train.shape)
print("Testing Sample Shape:",X_test.shape)

Training Sample Shape: (42000, 1, 28, 28)
Testing Sample Shape: (28000, 1, 28, 28)


### Performing One Hot encoding:

In [13]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(Y_train, 10)

In [14]:
print(X_train.shape)
print(X_test.shape)

(42000, 1, 28, 28)
(28000, 1, 28, 28)


### Network and training

In [15]:
# network and training
NB_EPOCH = 100
BATCH_SIZE = 256
VERBOSE = 1
OPTIMIZER = Adam()
VALIDATION_SPLIT=0.2
IMG_ROWS, IMG_COLS = 28, 28 # input image dimensions
NB_CLASSES = 10 # number of outputs = number of digits
INPUT_SHAPE = (1, IMG_ROWS, IMG_COLS)


### LeNet Code in Keras:

In [19]:
#define the ConvNet
class LeNet:
    @staticmethod
    def build(input_shape, classes):
        model = Sequential() # Sequential model
     
        # CONV => RELU => POOL
        
        model.add(Conv2D(20, kernel_size=5, padding="same",input_shape=input_shape))
        # 20 ->  Convolutional kernel
        # kernel_size = 5 --> Specify the value of spatial dimensions.
        # padding ='same' --> We have an output same size as the input.
        # input_shape --> input shape of the image
        
        model.add(Activation("relu"))
        # Activation --> Relu
        model.add(Conv2D(50, kernel_size=5, border_mode="same", activation='relu'))
        
        
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # pool_size = (2,2) --> Represents factors in which the image is vertically and horizontally downscaled.
        model.add(Dropout(0.25))
        
       
        # CONV => RELU => POOL
        model.add(Conv2D(50, kernel_size=5, border_mode="same"))
        model.add(Activation("relu"))
        model.add(Conv2D(50, kernel_size=5, border_mode="same",activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Dropout(0.25))
        
        model.add(Conv2D(50, kernel_size=5, border_mode="same"))
        model.add(Activation("relu"))
        model.add(Conv2D(50, kernel_size=5,border_mode="same", activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        model.add(Dropout(0.25))
        
        
        # Flatten => RELU layers   
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))
        model.add(Dropout(0.5))
        
        # a softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        return model

### Designing the neural network architecture:


In [20]:
# initialize the optimizer and model
model = LeNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)
model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
metrics=["accuracy"])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, kernel_size=5, activation="relu", padding="same")`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, kernel_size=5, padding="same")`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, kernel_size=5, activation="relu", padding="same")`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, kernel_size=5, padding="same")`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, kernel_size=5, activation="relu", padding="same")`


### Training:

In [ ]:
history = model.fit(X_train, Y_train,
batch_size=BATCH_SIZE, epochs=NB_EPOCH,
verbose=VERBOSE, validation_split=VALIDATION_SPLIT)

Train on 33600 samples, validate on 8400 samples
Epoch 1/100
33600/33600 [==============================] - 1606s 48ms/step - loss: 0.4178 - acc: 0.8581 - val_loss: 0.0786 - val_acc: 0.9750
Epoch 2/100
33600/33600 [==============================] - 1624s 48ms/step - loss: 0.0864 - acc: 0.9746 - val_loss: 0.0436 - val_acc: 0.9855
Epoch 3/100
33600/33600 [==============================] - 1821s 54ms/step - loss: 0.0608 - acc: 0.9824 - val_loss: 0.0410 - val_acc: 0.9877
Epoch 4/100
33600/33600 [==============================] - 1642s 49ms/step - loss: 0.0503 - acc: 0.9854 - val_loss: 0.0354 - val_acc: 0.9898
Epoch 5/100
33600/33600 [==============================] - 1749s 52ms/step - loss: 0.0412 - acc: 0.9877 - val_loss: 0.0298 - val_acc: 0.9914
Epoch 6/100
33600/33600 [==============================] - 1823s 54ms/step - loss: 0.0362 - acc: 0.9894 - val_loss: 0.0392 - val_acc: 0.9892
Epoch 7/100
33600/33600 [==============================] - 1651s 49ms/step - loss: 0.0324 - acc: 0.9902 -

In [ ]:
predictions = model.predict_classes(X_test, verbose=0)
submissions=pd.DataFrame({'ImageId':list(range(1,len(predictions) + 1)), "Label": predictions})
submissions.to_csv("Final_Prediction_2.csv", index=False, header=True)